

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/NER_TRAFFIC_DE.ipynb)




# **Detect entities related to road traffic using pretrained NER model.**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens.
Otherwise, you can look at the example outputs at the bottom of the notebook.



## 1. Colab Setup

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.8
Spark NLP_JSL Version : 4.2.8


## 2. Select the NER model and construct the pipeline

In [4]:
MODEL_NAME = "ner_traffic"

**Create the pipeline**

In [5]:
document_assembler = DocumentAssembler() \
    .setInputCol('text')\
    .setOutputCol('document')

sentence_detector = SentenceDetector() \
    .setInputCols(['document'])\
    .setOutputCol('sentence')

tokenizer = Tokenizer()\
    .setInputCols(['sentence']) \
    .setOutputCol('token')

word_embeddings = WordEmbeddingsModel.pretrained("w2v_cc_300d", "de", "clinical/models") \
    .setInputCols(['sentence', 'token']) \
    .setOutputCol('embeddings')

clinical_ner = MedicalNerModel.pretrained(MODEL_NAME, "de", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverter()\
    .setInputCols(['sentence', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(
    stages=[
        document_assembler, 
        sentence_detector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter
        ])


w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
ner_traffic download started this may take some time.
[OK!]


## 3. Create example inputs

In [6]:
import pandas as pd 
sample_text = [
    """herbert neubauer der handelskonzern spar, der von den wettbewerbshütern bereits das okay für die übernahme von 27 ehemaligen zielpunkt - filialen erhalten hat , hat nach gesprächen mit den vermietern die übernahme von 25 filialen fixiert. dies sagte spar - vorstand hans reisch am dienstag am rande eines pressegespräches im gespräch mit der apa. der umbau wird - sobald wir die genehmigungen erhalten - schritt für schritt erfolgen, so reisch. bis jahresende soll der umbau aller übernommenen filialen abgeschlossen sein . die ungarn - tochter, die in den letzten jahren noch millionenverluste geschrieben hat, ist laut reisch im vergangenen jahr wieder in die schwarzen zahlen zurückgekehrt. das letzte jahr war operativ toll, sagte reisch. schuld a den millionenverlusten sei die vom ungarischen staat eingehobene kr.""",
    
    """Serviceplan verteidigt die spitzenposition publips ist a den standorten madrid und valencia vertreten und beschäftigt insgesamt 60 mitarbeiter. In madrid soll langfristig auch ein haus der kommunikation mit den drei marken serviceplan, plan.net und mediaplus entstehen. Angaben zum kaufpreis und zum honorarvolumen der neuen agentur machen die verantwortlichen nicht. Zu den kunden von publips gehören die unternehmen ikea, amstel, makro und jack & jones. für metro / makro und bahlsen wurden bereits erste projekte zusammen mit der serviceplan - gruppe realisiert. gegenüber anderen spanischen agenturen dieser größe weist publips eine besonderheit auf: die von luis piquer geführte firma verfügt über ein eigenes regionales netzwerk mit standorten auf den balearen und kanaren, in katalonien sowie in andalusien. regionale aussteuerung ist in spanien sehr wichtig, erläutert markus noder, geschäftsführer bei serviceplan international, dies.""",
    
    """a92 deggendorf richtung münchen zwischen altdorf und moosburg - nord 5 km stockender verkehr""",
    
    """großartig, jetzt schon 15 min. verspätung - und es wird noch ein außerplanmäßiger halt eingeplant. soviel zu früher nachhause. #sbb""",
    
    """Karlsruhe richtung stuttgart zwischen leonberg - west und dreieck leonberg gegenstände auf der fahrbahn -lrb- ein reifenmantel -rrb-""",    
]
df = spark.createDataFrame(pd.DataFrame({'text': sample_text}))
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|herbert neubauer der handelskonzern spar, der von den wettbewerbshütern bereits das okay für die ...|
|Serviceplan verteidigt die spitzenposition publips ist a den standorten madrid und valencia vertr...|
|        a92 deggendorf richtung münchen zwischen altdorf und moosburg - nord 5 km stockender verkehr|
|großartig, jetzt schon 15 min. verspätung - und es wird noch ein außerplanmäßiger halt eingeplant...|
|Karlsruhe richtung stuttgart zwischen leonberg - west und dreieck leonberg gegenstände auf der fa...|
+----------------------------------------------------------------------------------------------------+



## 4. Use the pipeline to create outputs

In [7]:
from pyspark.sql.types import StringType, IntegerType

df = spark.createDataFrame(sample_text, StringType()).toDF("text")
result = nlp_pipeline.fit(df).transform(df)

In [8]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+----------------+--------------------+
|chunk           |ner_label           |
+----------------+--------------------+
|herbert neubauer|PERSON              |
|spar            |ORGANIZATION_COMPANY|
|übernahme       |TRIGGER             |
|27              |NUMBER              |
|zielpunkt       |ORGANIZATION_COMPANY|
|25              |NUMBER              |
|spar            |ORGANIZATION_COMPANY|
|vorstand        |ORG_POSITION        |
|hans reisch     |PERSON              |
|dienstag        |DATE                |
|apa             |ORGANIZATION        |
|jahresende      |DATE                |
|ungarn          |LOCATION            |
|reisch          |PERSON              |
|vergangenen jahr|DATE                |
|letzte jahr     |DATE                |
|reisch          |PERSON              |
|ungarischen     |LOCATION            |
|Serviceplan     |ORGANIZATION_COMPANY|
|publips         |ORGANIZATION_COMPANY|
+----------------+--------------------+
only showing top 20 rows



## 5. Visualize results

In [9]:
sample_text[0]

'herbert neubauer der handelskonzern spar, der von den wettbewerbshütern bereits das okay für die übernahme von 27 ehemaligen zielpunkt - filialen erhalten hat , hat nach gesprächen mit den vermietern die übernahme von 25 filialen fixiert. dies sagte spar - vorstand hans reisch am dienstag am rande eines pressegespräches im gespräch mit der apa. der umbau wird - sobald wir die genehmigungen erhalten - schritt für schritt erfolgen, so reisch. bis jahresende soll der umbau aller übernommenen filialen abgeschlossen sein . die ungarn - tochter, die in den letzten jahren noch millionenverluste geschrieben hat, ist laut reisch im vergangenen jahr wieder in die schwarzen zahlen zurückgekehrt. das letzte jahr war operativ toll, sagte reisch. schuld a den millionenverlusten sei die vom ungarischen staat eingehobene kr.'

In [10]:
from sparknlp_display import NerVisualizer

for i in range(result.count()):
  print("\n\n","*****"*40)
  print("\n",sample_text[i][:150],"...\n\n")
  NerVisualizer().display(
    result = result.collect()[i],
    label_col = 'ner_chunk',
    document_col = 'document'
  ) 



 ********************************************************************************************************************************************************************************************************

 herbert neubauer der handelskonzern spar, der von den wettbewerbshütern bereits das okay für die übernahme von 27 ehemaligen zielpunkt - filialen erha ...






 ********************************************************************************************************************************************************************************************************

 Serviceplan verteidigt die spitzenposition publips ist a den standorten madrid und valencia vertreten und beschäftigt insgesamt 60 mitarbeiter. In mad ...






 ********************************************************************************************************************************************************************************************************

 a92 deggendorf richtung münchen zwischen altdorf und moosburg - nord 5 km stockender verkehr ...






 ********************************************************************************************************************************************************************************************************

 großartig, jetzt schon 15 min. verspätung - und es wird noch ein außerplanmäßiger halt eingeplant. soviel zu früher nachhause. #sbb ...






 ********************************************************************************************************************************************************************************************************

 Karlsruhe richtung stuttgart zwischen leonberg - west und dreieck leonberg gegenstände auf der fahrbahn -lrb- ein reifenmantel -rrb- ...


